# Transfer Learning and Fine Tuning

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

#### Using GPU (highly recommended)

-> If using `theano` backend:

`THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32`

In [1]:
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from numpy import nan

now = datetime.datetime.now

Using TensorFlow backend.


### Settings

In [3]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [5]:
def train_model(model, train, test, nb_classes):
    
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

## Dataset Preparation

In [6]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

In [7]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

/srv/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", input_shape=(28, 28, 1...)`
  """
/srv/venv/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  import sys


In [8]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples


/srv/venv/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 21s 696us/step - loss: 0.1660 - acc: 0.9484 - val_loss: 0.0284 - val_acc: 0.9903
Epoch 2/5
30596/30596 [==============================] - 21s 694us/step - loss: 0.0496 - acc: 0.9856 - val_loss: 0.0147 - val_acc: 0.9949
Epoch 3/5
30596/30596 [==============================] - 21s 688us/step - loss: 0.0342 - acc: 0.9900 - val_loss: 0.0115 - val_acc: 0.9953
Epoch 4/5
30596/30596 [==============================] - 21s 680us/step - loss: 0.0274 - acc: 0.9922 - val_loss: 0.0080 - val_acc: 0.9971
Epoch 5/5
30596/30596 [==============================] - 21s 679us/step - loss: 0.0230 - acc: 0.9932 - val_loss: 0.0075 - val_acc: 0.9977
Training time: 0:01:45.532555
Test score: 0.0075334937268
Test accuracy: 0.997664915353


In [9]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples


/srv/venv/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 11s 380us/step - loss: 0.2320 - acc: 0.9318 - val_loss: 0.0488 - val_acc: 0.9852
Epoch 2/5
29404/29404 [==============================] - 11s 361us/step - loss: 0.0790 - acc: 0.9759 - val_loss: 0.0332 - val_acc: 0.9899
Epoch 3/5
29404/29404 [==============================] - 11s 364us/step - loss: 0.0608 - acc: 0.9803 - val_loss: 0.0298 - val_acc: 0.9907
Epoch 4/5
29404/29404 [==============================] - 11s 364us/step - loss: 0.0492 - acc: 0.9850 - val_loss: 0.0264 - val_acc: 0.9905
Epoch 5/5
29404/29404 [==============================] - 11s 365us/step - loss: 0.0444 - acc: 0.9867 - val_loss: 0.0241 - val_acc: 0.9918
Training time: 0:00:54.232421
Test score: 0.0241187664686
Test accuracy: 0.991771240375


## Your Turn

Try to Fine Tune a VGG16 Network

In [10]:
## your code here

```python
...
...
# Plugging new Layers
    model.add(Dense(768, activation='sigmoid'))
    model.add(Dropout(0.0))
    model.add(Dense(768, activation='sigmoid'))
    model.add(Dropout(0.0))
    model.add(Dense(n_labels, activation='softmax'))
```